## Binance top 200 coins by volume and momentum

In [1]:
coins = 3

## 1- Obtengo la data usando la API de Binance

In [2]:
import requests
import pandas as pd
from datetime import datetime, timedelta

def get_top_volume_coins_list_api(count=200):
    # Obtener la lista de todos los símbolos de trading
    symbols_url = "https://api.binance.com/api/v3/exchangeInfo"
    symbols_response = requests.get(symbols_url)
    symbols_data = symbols_response.json()
    
    # Filtrar solo los pares con USDT
    usdt_symbols = [symbol['symbol'] for symbol in symbols_data['symbols'] if symbol['symbol'].endswith('USDT')]

    # Elimino las stablecoins
    stablecoins = ['USDCUSDT', 'FDUSDUSDT', '1000SATS', '1000SATSBTC']
    usdt_symbols = [symbol for symbol in usdt_symbols if symbol not in stablecoins]
    
    # Calcular la fecha de inicio (7 días atrás)
    start_time = int((datetime.now() - timedelta(days=7)).timestamp() * 1000)
    
    # Obtener el volumen para cada símbolo
    volumes = []
    for symbol in usdt_symbols:
        klines_url = f"https://api.binance.com/api/v3/klines?symbol={symbol}&interval=1d&startTime={start_time}"
        klines_response = requests.get(klines_url)
        klines_data = klines_response.json()
        
        # Calcular el volumen promedio
        total_volume = sum(float(kline[7]) for kline in klines_data) #7 es quote asset volume
        avg_volume = total_volume / len(klines_data) if klines_data else 0
        volumes.append((symbol[:-4], avg_volume))  # Removemos 'USDT' del símbolo
    
    # Ordenar por volumen descendente y tomar los primeros 'count'
    top_volumes = sorted(volumes, key=lambda x: x[1], reverse=True)[:count]
    
    return top_volumes  # Ahora retornamos la lista completa de tuplas (moneda, volumen)

def get_top_volume_coins_df_api(count=200):
    volumes = get_top_volume_coins_list_api(count)
    df = pd.DataFrame(volumes, columns=['coin', 'average volume last 7 days'])

    # Ordenar el DataFrame antes de formatear los números
    df = df.sort_values('average volume last 7 days', ascending=False)

   # Formatear los números después de ordenar
    df['average volume last 7 days'] = df['average volume last 7 days'].apply(lambda x: f'{x:.2f}')

    return df

/Users/egrados/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
api_top_coins_list = get_top_volume_coins_list_api(coins)
#api_top_coins_list

In [4]:
api_top_coins_df = get_top_volume_coins_df_api(coins)
api_top_coins_df

,coin,average volume last 7 days
0,BTC,1426550813.55
1,ETH,644129078.29
2,SOL,357015751.11


#### Conclusion
Mejor uso el API directamente, ya que evito que CS50 tenga que instalar un paquete adicional de python. 

Las variables que pasan a la siguiente etapa son: 


api_top_coins_df 

api_top_coins_list

Trabajare con la lista pero primero tengo que eliminar los valores de volumen, que hago a continuacion

In [5]:
# Get the ticker list for coins.
top_coins = [item[0] for item in api_top_coins_list]
top_coins

['BTC', 'ETH', 'SOL']

## 2- Obtener klines para cada criptomoeda

#### USANDO EL WRAPER

In [6]:
# Obtiene klines con la lista top_coins, el par BTC y el par USDT
# FUNCION QUE USA EL WRAPER
import pandas as pd
from binance.client import Client
from datetime import datetime, timedelta, timezone
import os


client = Client()

not_available_btcpair = []
# FUNCION PARA BTC PAIRS
def get_klines_btcpair(crypto_list, weeks_back: int):
    """
    loopback: número de días a retroceder desde la fecha de referencia
    reference_low: fecha de referencia en formato "dd-mm-yyyy"
    
    weeks_back: int que indica la cantidad de semanas atras a considerar. 0 para esta semana, 1 para la semana anterior, etc.
    """
    today = datetime.now(timezone.utc)  # Asegurarse de trabajar en UTC
    
    # Calcular el lunes de la semana actual
    monday_this_week = today - timedelta(days=today.weekday())

    # Retroceder las semanas indicadas
    target_monday = monday_this_week - timedelta(weeks=weeks_back)

    # Establecer la hora a 00:00 en UTC
    target_monday_at_midnight = target_monday.replace(hour=0, minute=0, second=0, microsecond=0)

    # Retornar como timestamp en UTC en str
    start_timestamp = str(target_monday_at_midnight.timestamp())
    print(f"Start timestamp: {start_timestamp}")


    dataframes = []

    # Crear la carpeta 'data' si no existe
    if not os.path.exists('data'):
        os.makedirs('data')

    for crypto in crypto_list:
        try:
            print(f"Fetching data for {crypto}BTC...")
            klines = client.get_historical_klines(
                symbol=f"{crypto}BTC", # for BTC pairs
                #symbol=f"{crypto}USDT", # for USDT pairs
                interval=Client.KLINE_INTERVAL_1DAY,
                start_str=start_timestamp
            )

            # Crear el DataFrame
            df = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
            
            # Convertir el timestamp a datetime
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            
            # Establecer el timestamp como índice
            df.set_index('timestamp', inplace=True)
            
            # Convertir las columnas necesarias a float
            for col in ['open', 'high', 'low', 'close', 'volume']:
                df[col] = df[col].astype(float)
            
            # Guardar el DataFrame como CSV
            filename = f"data/{crypto}BTC_df.csv"
            df.to_csv(filename)
            
            dataframes.append(df)
            print(f"Datos de {crypto} guardados en {filename}")

        except Exception as e:
            print(f"Error al obtener datos para {crypto}BTC.")
            print(e)
            not_available_btcpair.append(crypto)
            continue

    return dataframes

#ejemplo de uso
dataframes = get_klines_btcpair(top_coins, 1)

Start timestamp: 1725235200.0
Fetching data for BTCBTC...
Error al obtener datos para BTCBTC.
APIError(code=-1121): Invalid symbol.
Fetching data for ETHBTC...
Datos de ETH guardados en data/ETHBTC_df.csv
Fetching data for SOLBTC...
Datos de SOL guardados en data/SOLBTC_df.csv


In [7]:
not_available_btcpair

['BTC']

#### USANDO LA API PELADA

In [8]:
# Obtiene klines con la lista top_coins, el par BTC y el par USDT
# FUNCION QUE USA EL WRAPER
import pandas as pd
from binance.client import Client
from datetime import datetime, timedelta, timezone
import os


client = Client()

not_available_btcpair = []
# FUNCION PARA BTC PAIRS
def get_klines_btcpair_api(crypto_list, weeks_back: int):
    """
    loopback: número de días a retroceder desde la fecha de referencia
    reference_low: fecha de referencia en formato "dd-mm-yyyy"
    
    weeks_back: int que indica la cantidad de semanas atras a considerar. 0 para esta semana, 1 para la semana anterior, etc.
    """
    today = datetime.now(timezone.utc)  # Asegurarse de trabajar en UTC
    
    # Calcular el lunes de la semana actual
    monday_this_week = today - timedelta(days=today.weekday())

    # Retroceder las semanas indicadas
    target_monday = monday_this_week - timedelta(weeks=weeks_back)

    # Establecer la hora a 00:00 en UTC
    target_monday_at_midnight = target_monday.replace(hour=0, minute=0, second=0, microsecond=0)

    # Retornar como timestamp en UTC en str
    start_timestamp = str(target_monday_at_midnight.timestamp())


    dataframes = []

    # Crear la carpeta 'data' si no existe
    if not os.path.exists('data'):
        os.makedirs('data')

    for crypto in crypto_list:
        try:
            print(f"Fetching data for {crypto}BTC...")
            klines = client.get_historical_klines(
                symbol=f"{crypto}BTC", # for BTC pairs
                #symbol=f"{crypto}USDT", # for USDT pairs
                interval=Client.KLINE_INTERVAL_1DAY,
                start_str=start_timestamp
            )

            # Crear el DataFrame
            df = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
            
            # Convertir el timestamp a datetime
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            
            # Establecer el timestamp como índice
            df.set_index('timestamp', inplace=True)
            
            # Convertir las columnas necesarias a float
            for col in ['open', 'high', 'low', 'close', 'volume']:
                df[col] = df[col].astype(float)
            
            # Guardar el DataFrame como CSV
            filename = f"data/{crypto}BTC_df.csv"
            df.to_csv(filename)
            
            dataframes.append(df)
            print(f"Datos de {crypto} guardados en {filename}")

        except Exception as e:
            print(f"Error al obtener datos para {crypto}BTC.")
            print(e)
            not_available_btcpair.append(crypto)
            continue

    return dataframes

#ejemplo de uso
#dataframes = get_klines_btcpair(top_coins, 1)

## Analisis core de las monedas mas importantes de Binance

In [10]:
import pandas as pd
import os
from datetime import datetime

def analyze_momentum(pivot_low_date, reference_high_date):
    # Convertir las fechas de string a datetime, fechas en formato DD-MM-YYYY
    pivot_low_date = datetime.strptime(pivot_low_date, "%d-%m-%Y")
    reference_high_date = datetime.strptime(reference_high_date, "%d-%m-%Y")

    results = []

    # Leer todos los archivos CSV en la carpeta 'data'
    for filename in os.listdir('data'):
        print(filename)

        if filename.endswith('.csv'):
            coin = filename.split('_')[0]  # Extraer el nombre de la moneda del nombre del archivo
            df = pd.read_csv(os.path.join('data', filename), index_col='timestamp', parse_dates=True)

            # Asegurarse de que las columnas necesarias son de tipo float
            for col in ['low', 'high']:
                df[col] = df[col].astype(float)

            # Manejo de excepciones si no se encuentra la fecha
            try:

                # Encontrar el pivot_low
                pivot_low = df.loc[pivot_low_date.strftime("%Y-%m-%d"), 'low']

                # Encontrar el max_high después del pivot_low_date
                max_high = df.loc[pivot_low_date:, 'high'].max()

                # Encontrar el reference_high
                reference_high = df.loc[reference_high_date.strftime("%Y-%m-%d"), 'high']

                # Calcular el cambio porcentual
                change_pct = ((reference_high - pivot_low) / pivot_low) * 100

                results.append({
                    'coin': coin,
                    'pivot_low': pivot_low,
                    'max_high': max_high,
                    'reference_high': reference_high,
                    'change_pct': change_pct
                })
            except KeyError:
                print (f"se ha encontrado un keyerror para {filename}")

    # Crear el DataFrame final
    result_df = pd.DataFrame(results)

    # Ordenar el DataFrame por change_pct de mayor a menor
    result_df = result_df.sort_values('change_pct', ascending=False)

    return result_df

# Ejemplo de uso:
#aa_df = analyze_momentum(pivot_low_date="05-08-2024", reference_high_date="25-08-2024")
